In [1]:
%load_ext blackcellmagic

In [2]:
#!pipenv install python-decouple requests asyncio nest_asyncio aiohttp bs4 --dev
from decouple import config

In [3]:
import json
import requests
import sys
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
from itertools import zip_longest
import aiohttp
from aiohttp import ClientSession
import pandas as pd
from random import randint
nest_asyncio.apply()
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import re

In [4]:
CLIENT_SECRET= config("CLIENT_SECRET")
CLIENT_ID = config("CLIENT_ID")
CLIENT_ACCESS_TOKEN = config("CLIENT_ACCESS_TOKEN")

## Step One: Get all the URL's for each song

In [5]:
import requests


def request_song_info(session, song_num, song_urls):
    base_url = "https://api.genius.com/songs/" + str(song_num)
    headers = {"Authorization": "Bearer " + CLIENT_ACCESS_TOKEN}
    response = requests.get(base_url, headers=headers)
    try:
        if response.json()["meta"]["status"] == 200:
            song_urls.append(
                (
                    song_num,
                    response.json()["response"]["song"]["title"],
                    response.json()["response"]["song"]["url"],
                )
            )
        else:
            pass
    except Exception as e:
        print(song_num, e)

In [6]:
async def get_index_data_asynchronous(min_song, max_song, song_urls):
    """
    1. Establish an executor and number of workers
    2. Establish the session
    3. Establish the event loop
    4. Create the task by list comprenhensions
    5. Gather tasks.
    """
    with ThreadPoolExecutor(max_workers=40) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, request_song_info, *(session, song_num, song_urls)
                )
                for song_num in range(min_song, max_song)
            ]
            for response in await asyncio.gather(*tasks):
                pass


def execute_async_index_event_loop(min_song, max_song, song_urls):
    """
    This function does something analogous to compiling the get_data_asynchronously function,
    Then it executes loop.
    1. Call the get_data_function
    2. Get the event_loop
    3. Run the tasks (Much easier to understand in python 3.7, "ensure_future" was changed to "create_task")
    4. Edge_list and top_interactions will be passed to the next functions
    """
    future = asyncio.ensure_future(
        get_index_data_asynchronous(min_song, max_song, song_urls)
    )
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    return song_urls

## tests

In [7]:
# %%time
# test_song_urls = []
# min_song=0
# song_urls = execute_async_index_event_loop(min_song,100,test_song_urls)

In [12]:
len(test_song_urls)

91

## Running this iteratively
It takes about ~20 minutes per 100,000 queries. This algorithm does have poor error handling(it just skips some errors, but some raise exception and causes it to break), so the total ends up less than is expected. However, from working on this project a lot I know that many entries have been deleted and should explain the missing values.

In [11]:
song_urls = []

In [8]:
# %time
# #min_song = max([s[0] for s in song_urls])+1
# min_song = 0
# max_song = min_song +100000
# song_urls = execute_async_index_event_loop(min_song, max_song,song_urls)

In [10]:
len(song_urls)

93186

In [11]:
s = set(song_urls)
len(s)

93186

In [12]:
min_song = max([i[0] for i in list(s)])+1
min_song

100000

In [21]:
min_song = max([s[0] for s in song_urls])+1
min_song

200000

In [ ]:
%%time
#min_song =0
min_song = max([s[0] for s in song_urls])+1
max_song = min_song + 100000
song_urls = execute_async_index_event_loop(min_song, max_song,song_urls)

639997 Expecting value: line 1 column 1 (char 0)
640041 Expecting value: line 1 column 1 (char 0)


In [13]:
max_song

2000096

In [11]:
len(song_urls), len(set(song_urls))

(471875, 371797)

In [12]:
min_song = max([i[0] for i in list(song_urls)])+1
min_song

614845

## Pickling. 
Pickling the data after each loop.

In [47]:
import pickle
filename = "song_list_tuple"
outfile = open(filename,'wb')
pickle.dump(song_urls, outfile)
outfile.close()

In [9]:
import pickle
filename = "song_list_tuple"
infile = open(filename, "rb")
song_urls = pickle.load(infile)
infile.close()

In [10]:
song_urls

[(3, 'Can I Live', 'https://genius.com/Jay-z-can-i-live-lyrics'),
 (11, 'Lord You Know', 'https://genius.com/Camron-lord-you-know-lyrics'),
 (16, '’98 Freestyle', 'https://genius.com/Big-l-98-freestyle-lyrics'),
 (10,
  'Rockin’ and Rollin’',
  'https://genius.com/Camron-rockin-and-rollin-lyrics'),
 (7, 'Lollipop (Remix)', 'https://genius.com/Lil-wayne-lollipop-remix-lyrics'),
 (9, 'Family Ties', 'https://genius.com/Camron-family-ties-lyrics'),
 (15, 'December 4th', 'https://genius.com/Jay-z-december-4th-lyrics'),
 (1, 'Killa Cam', 'https://genius.com/Camron-killa-cam-lyrics'),
 (19,
  'Losing Weight, Pt. 2',
  'https://genius.com/Camron-losing-weight-pt-2-lyrics'),
 (13,
  'Think Ya’ll Know',
  'https://genius.com/Fabolous-think-yall-know-lyrics'),
 (12,
  'Money On My Mind',
  'https://genius.com/Lil-wayne-money-on-my-mind-lyrics'),
 (5, 'Down and Out', 'https://genius.com/Camron-down-and-out-lyrics'),
 (6, 'Fly In', 'https://genius.com/Lil-wayne-fly-in-lyrics'),
 (18,
  'It’s Hot (S

### Putting it all together

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import re


def get_page(session,song_tuple,songs_list):
    url = song_tuple[2]
    page = requests.get(url)
#     soup = BeautifulSoup(page.text, "html.parser")
    songs_list.append((song_tuple[0],song_tuple[1],song_tuple[2],page))



In [ ]:
async def get_index_data_asynchronous(songs_list, song_urls):
    """
    1. Establish an executor and number of workers
    2. Establish the session
    3. Establish the event loop
    4. Create the task by list comprenhensions
    5. Gather tasks.
    """
    with ThreadPoolExecutor(max_workers=40) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, get_page, *(session, song_tuple,songs_list)
                )
                for song_tuple in song_urls
            ]
            for response in await asyncio.gather(*tasks):
                pass


def execute_async_index_event_loop(songs_list, song_urls):
    """
    This function does something analogous to compiling the get_data_asynchronously function,
    Then it executes loop.
    1. Call the get_data_function
    2. Get the event_loop
    3. Run the tasks (Much easier to understand in python 3.7, "ensure_future" was changed to "create_task")
    4. Edge_list and top_interactions will be passed to the next functions
    """
    future = asyncio.create_task(
        get_index_data_asynchronous(songs_list, song_urls)
    )
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    filename = "song_lists/songs_list"+str(randint(0,100000000))
    outfile = open(filename,'wb')
    pickle.dump(songs_list1, outfile)
    outfile.close()

In [ ]:
for i in range(0,159,1):
    songs_list1=[]
    song_urls1 =song_urls[i*10000:i*10000+10000]
    execute_async_index_event_loop(songs_list1, song_urls1)

# extracting info from pages

In [ ]:
def get_text(response):
    """Get text from a requests response object"""
    r = response.text
    r = BeautifulSoup(r, "html.parser")
    return r


def get_lyrics(soup_text):
    """extract lyrics from the beautiful soup text"""
    try:
        lyrics = soup_text.find_all("p")[0].text
        lyrics = re.sub(r"\[.*?\']", "", lyrics)
        lyrics = re.sub(r"--", "", lyrics)
        lyrics = lyrics.split("\n")
        lyrics = [x for x in lyrics if x != ""]
        return lyrics
    except:
        np.nan


def get_side_table_rows(page):
    """extract the side tables from the beatiful soup text"""
    side_table = page.find_all(
        "div", {"class": "metadata_unit metadata_unit--table_row"}
    )
    side_table = [get_side_table_row(i) for i in side_table]
    return side_table


def get_side_table_row(side_list_table_row):
    a = side_list_table_row.find_all("span", {"class":"metadata_unit-label"})
    label = a[0].text
    links_list = side_list_table_row.find_all("a")
    links = [get_links(links_list,i) for i in range(len(links_list))]
    return label,links

def get_links(x, i):
    return x[i]["href"]

def get_header_links(page):
    """
    find unit containing the header info
    use 
    """
    header = page.findAll("div", {"class":"header_with_cover_art-primary_info" })
    header_links = [i.a["href"] for i in header ]
    return header_links

In [ ]:
import pickle
import os
pickle_list = os.listdir("song_lists/")

In [ ]:
pickle_list

In [ ]:
for i in pickle_list:
    #load the pickled tuple and convert to df
    filename = "song_lists/"+i
    outfile = open(filename,'rb')
    z = pickle.load(outfile)
    outfile.close()
    z = pd.DataFrame(z, columns = ["song_num","title","url","response"])

    #get text from response and drop response
    z["page"] = z["response"].apply(get_text)
    z = z.drop(["response"],axis=1)
    #Extract desired data
    z["lyrics"]=z["page"].apply(get_lyrics)
    z["side_table"] = z["page"].apply(get_side_table_rows)
    z["header_links"] = z["page"].apply(get_header_links)
    z = z.drop(["page"], axis=1)

    #repickle the model
    filename = "song_dfs/"+i
    outfile = open(filename,'wb')
    pickle.dump(z, outfile)
    outfile.close()

In [23]:
z

(10000, 7)

# Test

In [4]:
filename = "song_dfs/songs_list1077982"
outfile = open(filename,'rb')
w = pickle.load(outfile)
outfile.close()

In [5]:
w

,song_num,title,url,lyrics,side_table,header_links
0,109,The World Is Yours,https://genius.com/Nas-the-world-is-yours-lyrics,"[[Produced by Pete Rock], [Chorus: Pete Rock +...","[(Written By, [https://genius.com/artists/Pete...",[https://genius.com/artists/Nas]
1,129,Rain,https://genius.com/Chamillionaire-rain-lyrics,"[[Talking], I was sitting back, and I was thin...","[(Written By, [https://genius.com/artists/Scar...",[https://genius.com/artists/Chamillionaire]
2,118,Glass,https://genius.com/Pill-glass-lyrics,"[[Verse 1 - Pill], Every morning I wake up, I'...",[],[https://genius.com/artists/Pill]
3,123,It's Like That,https://genius.com/Jay-z-its-like-that-lyrics,"[[Intro: Jay-Z, Kid Capri & Both], Yeah, uh-hu...","[(Written By, [https://genius.com/artists/Fran...",[https://genius.com/artists/Jay-z]
4,104,Angelz,https://genius.com/Mf-doom-angelz-lyrics,"[[Intro: Ghostface Killah], Yea, that's right,...","[(Written By, [https://genius.com/artists/Mf-d...",[https://genius.com/artists/Mf-doom]
5,135,Real As It Gets,https://genius.com/Jay-z-real-as-it-gets-lyrics,"[[Intro: Jeezy], Yeah, see it's that, It's tha...","[(Written By, [https://genius.com/artists/The-...",[https://genius.com/artists/Jay-z]
6,107,Crunk Muzik,https://genius.com/The-diplomats-crunk-muzik-l...,"[[Intro], Yeah! Ay! Dip-Set! Come on, Black-ou...","[(Written By, [https://genius.com/artists/Camr...",[https://genius.com/artists/The-diplomats]
7,121,Neva Eva (Get On My Level),https://genius.com/Trillville-neva-eva-get-on-...,"[[Intro: Lil Jon], What, uh, What?! You can't!...","[(Written By, [https://genius.com/artists/Lil-...",[https://genius.com/artists/Trillville]
8,116,Keys Open Doors,https://genius.com/Clipse-keys-open-doors-lyrics,"[[Hook: Pusha T], Keys open doors, keys open d...","[(Written By, [https://genius.com/artists/No-m...",[https://genius.com/artists/Clipse]
9,130,Not Pussified,https://genius.com/Heene-boys-not-pussified-ly...,"[[Intro], ""I hate these pajamas! These pajamas...",[],[https://genius.com/artists/Heene-boys]


# Putting dataframes together

In [ ]:
import pickle
import os
pickle_list = os.listdir("song_dfs/")

In [4]:
#pickle_list = pickle_list[1:]

In [ ]:
pickle_list

In [ ]:
df = pd.DataFrame(columns=["song_num","title","url","lyrics","side_table","header_links"])

In [ ]:
for i in pickle_list:
    import pickle
    filename = i
    infile = open("song_dfs/"+filename, "rb")
    sub_df = pickle.load(infile)
    infile.close()
    df = df.append(sub_df)

In [ ]:
import pickle
filename = "million.5_song_df"
outfile = open(filename,'wb')
pickle.dump(df, outfile)
outfile.close()

In [ ]:
df.to_csv("millio.5_song.csv", index=False)

In [2]:
import pickle
import pandas
filename = "million.5_song_df"
infile = open(filename,'rb')
df = pickle.load(infile)
infile.close()

EOFError: Ran out of input